In [12]:
#Packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import preprocessing
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pylab import rcParams
import dill

ModuleNotFoundError: No module named 'dill'

In [ ]:
#dill.dump_session('notebook_env.db')

In [ ]:
#dill.load_session('notebook_env.db')

In [ ]:
#load data, change directory to your corresponding directories
#tag = pd.read_csv("C:/Users/cos00/Desktop/Nuclear/APAN5900/Tag Data Final.csv")
tag = pd.read_csv('~/Desktop/APAN5900/Tag Data Final.csv')

In [ ]:
np.shape(tag) #dimension

In [ ]:
tag.describe()  #summary of tag data

In [ ]:
tag.head()

In [ ]:
sum(tag['MI'].notnull()) #only 1920not null value for MI

In [ ]:
#convert time column from string to time
tag['Time'] = [datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in tag['Time'] ] 

In [ ]:
#extract tag entries with MI value
tag_MI = tag[tag['MI'].notnull()]

In [ ]:
tag_MI.head() #glimpse of new tag_MI

In [ ]:
np.shape(tag_MI) #dimension

In [ ]:
#------exploratory analysis-----------
#plot of MI
plt.hist(tag_MI['MI'], bins = 30)
plt.show()

In [ ]:
#scatter plot of some variables against MI
plt.scatter(tag_MI['MI'], tag_MI['P1:FC70104'])
plt.show()

In [ ]:
plt.scatter(tag_MI['MI'], tag_MI['P1:FC70113'])
plt.show()

In [ ]:
plt.scatter(tag_MI['MI'], tag_MI['P1:FC70116'])
plt.show()

In [ ]:
#number of observations for each product grade
fig=plt.gcf()
fig.set_size_inches(12,8)
tag_MI['ProdGrade'].value_counts().plot(kind='bar') #1203K is most dominant product grade
plt.show()

In [ ]:
#heat map column 2 to 21
tag_cor1 = tag.iloc[:,2:21]
sns.heatmap(tag_cor1.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':14})
fig=plt.gcf()
fig.set_size_inches(22,18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
#heat map column 22 to 41
tag_cor2 = tag.iloc[:,np.r_[2, 22:41]]
sns.heatmap(tag_cor2.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':14})
fig=plt.gcf()
fig.set_size_inches(22,18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
#heat map column 42 to 61
tag_cor3 = tag.iloc[:,np.r_[2, 42:61]]
sns.heatmap(tag_cor3.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':14})
fig=plt.gcf()
fig.set_size_inches(22,18)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
#time series plot of MI value
time = tag_MI['Time']
mi = tag_MI['MI']
plt.plot(time,mi)
fig=plt.gcf()
fig.set_size_inches(18,15)
plt.show()

In [ ]:
#take a close look at what is going on at the spike
tag_MI.loc[(tag_MI['Time'] >= '2018-11-10') & (tag_MI['Time'] <= '2018-11-11')] 
#spike occurs on 2018-11-10 19:45 and 19:45

In [ ]:
#take a close look at what is going on at the spike
tag_MI.loc[(tag_MI['Time'] >= '2019-05-19') & (tag_MI['Time'] <= '2019-06-10')]
#it seems like spike occurs in late May

In [ ]:
#zoom in left side of spike
tag_left = tag_MI.loc[(tag_MI['Time'] >= '2018-11-10 09:15:00') & (tag_MI['Time'] <= '2018-11-12')]
plt.plot(tag_left['Time'],tag_left['MI'])
fig=plt.gcf()
fig.set_size_inches(14,11)
axes = plt.gca()
axes.set_ylim([0,55])
plt.show()

In [ ]:
#MI mean value by different progrades
Prograde_MI = tag_MI.groupby(['ProdGrade']).mean() #check mean MI for each Prograde
plt.barh(Prograde_MI.index, Prograde_MI['MI'], color = 'orange')
plt.show()

In [ ]:
#The tallest spike is of product 4100N, which has mean MI of 7.5. It is clearly an outlier that needs to be removed.
#second tallest spike is of 1102KR with mean MI of less than 5, remove this as well
#They can be removed since neither of these product grades has too few values
tag_MI = tag_MI[(tag_MI['MI'] != 52.88) & (tag_MI['MI'] != 25.14)]


In [ ]:
#----time series analysis------

#preprocesing
tag_Time = tag_MI[['Time', 'MI']]
#tag_Time = tag_Time.groupby('Time')
tag_Time = tag_Time.set_index('Time')
tag_Time.index

In [ ]:
#mean o mi
MI_time = tag_Time['MI'].resample('MS').mean()
MI_time['2018':]

In [ ]:
#plot seasonality and residual
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(MI_time, freq = 11, model='additive')
fig = decomposition.plot()
plt.show() #no trend, this is not time series

In [ ]:
#--------test modeling-----------

#split train and test
train, test = train_test_split(tag_MI, test_size=0.3, random_state=42)


In [ ]:
np.shape(train)

In [ ]:

#pre-processing, use all numeric predicators
X_train = train.drop(["Time", "ProdGrade", "MI"],axis=1)
Y_train = train["MI"]
X_test = test.drop(["Time", "ProdGrade", "MI"],axis=1)

In [ ]:
#1. linear model with all predicators
lm_model = LinearRegression().fit(X_train, Y_train)

In [ ]:
#check accuracy
lm_model.score(X_train, Y_train) #check r squared value

In [ ]:
#predict
MI_lm = lm_model.predict(X_test)

In [ ]:
#check rmse
mean_squared_error(test['MI'], MI_lm)

In [ ]:
#how many value do we get right exactly if we round to nearest 1 decimal   
pred_round_lm = pd.DataFrame(list(np.round(MI_lm,1)), round(test['MI'],1))
pred_round_lm.columns = ['predicted']
np.shape(pred_round_lm.loc[pred_round_lm['predicted'] == 
                           pred_round_lm.index])[0]/(np.shape(test)[0])
#3% accuracy

In [ ]:
#2. linear model with some predicators (coef >= abs(0.1))
predicators_used = ['P1:FC70113','P1:FC70310','P1:FFC70106','P1:FR70106','P1:QIA701001', 'P1:PR70200', 'P1:R700RECYCLEB', 
                     'P1:TI70141', 'P1:TI70304','P1:TR70104','P1:TR70201','P1:TR70305']
X_train_lm = X_train[predicators_used]
X_test_lm = X_test[predicators_used]
lm_model2 = LinearRegression().fit(X_train_lm, Y_train)

In [ ]:
#check accuracy
lm_model2.score(X_train_lm, Y_train) #check r squared value

In [ ]:
#predict
MI_lm2 = lm_model2.predict(X_test_lm)
#check rmse
mean_squared_error(test['MI'], MI_lm2)

In [3]:
#how many value do we get right exactly if we round to nearest 1 decimal   
pred_round_lm2 = pd.DataFrame(list(np.round(MI_lm2,1)), round(test['MI'],1))
pred_round_lm2.columns = ['predicted']
np.shape(pred_round_lm2.loc[pred_round_lm2['predicted'] == 
                           pred_round_lm2.index])[0]/(np.shape(test)[0])
#4.9% accuracy

NameError: name 'MI_lm2' is not defined

In [4]:
#3. Linear model with some predicators plus their interactions
tag_cor = tag.corr() #all correlations
tag_cor = tag_cor[predicators_used].loc[predicators_used] #subset ronly predictors we used (>= |0.2|)


NameError: name 'tag' is not defined

In [ ]:
tag_cor

In [ ]:
#print interactions
for index, row in tag_cor.iterrows():
    colinear = row[(row >= abs(0.4)) & (row < 1)]
    print(index + ": ")
    print(colinear.index)
            

In [ ]:
#combine interactions to our column
tag_MI_int = tag_MI
tag_MI_int['P1:FC70310 x P1:FR70106'] = tag_MI_int['P1:FC70310'] * tag_MI_int['P1:FR70106']
tag_MI_int['P1:PR70200 x P1:TR70305'] = tag_MI_int['P1:PR70200'] * tag_MI_int['P1:TR70305']
tag_MI_int['P1:R700RECYCLEB x P1:FR70106'] = tag_MI_int['P1:R700RECYCLEB'] * tag_MI_int['P1:FR70106']
tag_MI_int['P1:R700RECYCLEB x P1:TR70201'] = tag_MI_int['P1:R700RECYCLEB'] * tag_MI_int['P1:TR70201']
tag_MI_int['P1:R700RECYCLEB x P1:TR70305'] = tag_MI_int['P1:R700RECYCLEB'] * tag_MI_int['P1:TR70305']
tag_MI_int['P1:TI70141 x P1:TI70304'] = tag_MI_int['P1:TI70141'] * tag_MI_int['P1:TI70304']
tag_MI_int['P1:TI70141 x P1:TR70201'] = tag_MI_int['P1:TI70141'] * tag_MI_int['P1:TR70201']
tag_MI_int['P1:TI70141 x P1:TR70305'] = tag_MI_int['P1:TI70141'] * tag_MI_int['P1:TR70305']  
tag_MI_int['P1:TI70304 x P1:TR70201'] = tag_MI_int['P1:TI70304'] * tag_MI_int['P1:TR70201'] 
tag_MI_int['P1:TI70304 x P1:TR70305'] = tag_MI_int['P1:TI70304'] * tag_MI_int['P1:TR70305'] 
tag_MI_int['P1:TR70201 x P1:TR70305'] = tag_MI_int['P1:TR70201'] * tag_MI_int['P1:TR70305']

In [ ]:
#modeling
interactions_used = ['P1:FC70310 x P1:FR70106','P1:PR70200 x P1:TR70305','P1:R700RECYCLEB x P1:FR70106','P1:R700RECYCLEB x P1:TR70201',
                    'P1:R700RECYCLEB x P1:TR70305','P1:TI70141 x P1:TI70304','P1:TI70141 x P1:TR70201','P1:TI70141 x P1:TR70305',
                    'P1:TI70304 x P1:TR70201','P1:TI70304 x P1:TR70305','P1:TR70201 x P1:TR70305']
#preprocess
train_int, test_int = train_test_split(tag_MI_int, test_size=0.3, random_state=42)
X_train_int = train_int.drop(["Time", "ProdGrade", "MI"],axis=1)
Y_train_int = train_int["MI"]
X_test_int = test_int.drop(["Time", "ProdGrade", "MI"],axis=1)
X_train_int_lm = X_train_int[predicators_used + interactions_used]
X_test_int_lm = X_test_int[predicators_used + interactions_used]
#model
lm_model3 = LinearRegression().fit(X_train_int_lm, Y_train_int)

In [ ]:
#check accuracy
lm_model3.score(X_train_int_lm, Y_train_int) #check r squared value

In [ ]:
#predict
MI_lm3 = lm_model3.predict(X_test_int_lm)
#check rmse
mean_squared_error(test_int['MI'], MI_lm3)

In [ ]:
#how many value do we get right exactly if we round to nearest 1 decimal   
pred_round_lm3 = pd.DataFrame(list(np.round(MI_lm3,1)), round(test['MI'],1))
pred_round_lm3.columns = ['predicted']
np.shape(pred_round_lm3.loc[pred_round_lm3['predicted'] == 
                           pred_round_lm3.index])[0]/(np.shape(test)[0])
#6.6% accuracy

In [ ]:
#4. linear regression with normalization
X_train_norm = preprocessing.normalize(X_train)
lm_model4 = LinearRegression().fit(X_train_norm, Y_train)
#check accuracy
lm_model4.score(X_train_norm, Y_train) #check r squared value

In [ ]:
#predict
X_test_norm = preprocessing.normalize(X_test)
MI_lm4 = lm_model4.predict(X_test_norm)
#check rmse
mean_squared_error(test['MI'], MI_lm4)

In [ ]:
#how many value do we get right exactly if we round to nearest 1 decimal   
pred_round_lm4 = pd.DataFrame(list(np.round(MI_lm4,1)), round(test['MI'],1))
pred_round_lm4.columns = ['predicted']
np.shape(pred_round_lm4.loc[pred_round_lm4['predicted'] == 
                           pred_round_lm4.index])[0]/(np.shape(test)[0])
#4% accuracy

In [ ]:
# 5. linear regression of normalization with selected variables
X_train_nm = X_train.apply(lambda x: (x - x.min()) / (x.max() - x.min())) #mannual normalization for each column
X_train_nm['MI'] = train['MI']
MI_cor = X_train_nm.corr()['MI']
pred_used = MI_cor[(MI_cor >= abs(0.1)) & (MI_cor < 1)].index #predictors with >= abs0.1


In [5]:
X_train_nm = X_train_nm[pred_used]
lm_model_nm = LinearRegression().fit(X_train_nm, Y_train)
#check accuracy
lm_model_nm.score(X_train_nm, Y_train) #check r squared value


NameError: name 'X_train_nm' is not defined

In [6]:
#predict
X_test_nm = X_test.apply(lambda x: (x - x.min()) / (x.max() - x.min())) #mannual normalization for each column
X_test_nm = X_test[pred_used]
MI_lm_nm = lm_model_nm.predict(X_test_nm)
#check rmse
mean_squared_error(test['MI'], MI_lm_nm)

NameError: name 'X_test' is not defined

In [7]:
#6. linear regression with standardize
X_train_scale = preprocessing.scale(X_train)
lm_model5 = LinearRegression().fit(X_train_scale, Y_train)
#check accuracy
lm_model5.score(X_train_scale, Y_train) #check r squared value

NameError: name 'X_train' is not defined

In [8]:
#predict
X_test_scale = preprocessing.scale(X_test)
MI_lm5 = lm_model5.predict(X_test_scale)
#check rmse
mean_squared_error(test['MI'], MI_lm5)

NameError: name 'X_test' is not defined

In [9]:
#how many value do we get right exactly if we round to nearest 1 decimal   
pred_round_lm5 = pd.DataFrame(list(np.round(MI_lm5,1)), round(test['MI'],1))
pred_round_lm5.columns = ['predicted']
np.shape(pred_round_lm5.loc[pred_round_lm5['predicted'] == 
                           pred_round_lm5.index])[0]/(np.shape(test)[0])
#4.7% accuracy

NameError: name 'MI_lm5' is not defined

In [ ]:
#7. random forest
random_forest = RandomForestRegressor(n_estimators=500)
random_forest.fit(X_train, Y_train)
random_forest.score(X_train, Y_train)

In [ ]:
#checkaccuracy
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [ ]:
#predict
MI_rf = random_forest.predict(X_test)

In [ ]:
#check rmse
mean_squared_error(test['MI'], MI_rf)

In [ ]:
#how many value do we get right exactly if we round to nearest 1 decimal   
pred_round_rf = pd.DataFrame(list(np.round(MI_rf,1)), round(test['MI'],1))
pred_round_rf.columns = ['predicted']
np.shape(pred_round_lm3.loc[pred_round_rf['predicted'] == 
                           pred_round_rf.index])[0]/(np.shape(test)[0])
#10.2% accuracy

In [ ]:
#importance plot
feat_importances = pd.Series(random_forest.feature_importances_, index=X_train.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
#8. random forest with selected variables from importance plot 
X_train_rm = X_train[['P1:FFC70113', 'P1:FR70106','P1:FFC70106','P1:QIA701001','P1:FC70113']]
X_test_rm = X_test[['P1:FFC70113', 'P1:FR70106','P1:FFC70106','P1:QIA701001','P1:FC70113']]
random_forest.fit(X_train_rm, Y_train)
acc_random_forest = round(random_forest.score(X_train_rm, Y_train) * 100, 2)
acc_random_forest

In [ ]:
#predict
MI_rf2 = random_forest.predict(X_test_rm)
#check rmse
mean_squared_error(test['MI'], MI_rf2)

In [ ]:
#how many value do we get right exactly if we round to nearest 1 decimal
pred_round_rf2 = pd.DataFrame(list(np.round(MI_rf2,1)), round(test['MI'],1))
pred_round_rf2.columns = ['predicted']
np.shape(pred_round_rf.loc[pred_round_rf2['predicted'] == 
                           pred_round_rf2.index])[0]/(np.shape(test)[0])
#9.2 accuracy

In [ ]:
#9. boosting
boosting = GradientBoostingRegressor(n_estimators=500, max_depth=2)
boosting.fit(X_train, Y_train)
boosting.score(X_train, Y_train)

In [ ]:
#predict
MI_gb = boosting.predict(X_test)
#check rmse
mean_squared_error(test['MI'], MI_gb)

In [ ]:
#how many value do we get right exactly if we round to nearest 1 decimal
pred_round_gb = pd.DataFrame(list(np.round(MI_gb,1)), round(test['MI'],1))
pred_round_gb.columns = ['predicted']
np.shape(pred_round_gb.loc[pred_round_gb['predicted'] == 
                           pred_round_gb.index])[0]/(np.shape(test)[0])
#9.4 accuracy